In [ ]:
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
x_sheet = pd.read_excel("../data/coded_data.xlsx", usecols=[1, 2, 3, 4, 9, 10, 11])
x_sheet = x_sheet.fillna(0)

df = np.asmatrix(x_sheet.values)
y_sheet = pd.read_excel("../data/coded_data.xlsx", usecols=[7])
labels = np.array(y_sheet.values)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df, labels, test_size=0.2, random_state=89
)

In [ ]:
def mean_pred(y_true, y_pred):
    RMSE = tf.sqrt(tf.losses.mean_squared_error(y_true, y_pred))
    return RMSE

In [ ]:
model = tf.keras.models.load_model(
    "../models/model_current.h5", custom_objects={"mean_pred": mean_pred}
)

In [ ]:
i = 4
x_predict = df[i]

print(df[i])
print(labels[i])
print(model.predict(x_predict))

In [ ]:
test_ans = model.predict(X_test)

In [ ]:
tensor_RMSE = math.sqrt(mean_squared_error(y_test, test_ans))
print("RMSE: " + str(tensor_RMSE))

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(y_test, y_test)

In [ ]:
r2_score = linear_reg.score(y_test, test_ans)
print("R2 Score from RxR + Scatter Predicted: %f" % r2_score)

slope, intercept, r_value, p_value, std_err = stats.linregress(
    y_test[:, 0], test_ans[:, 0]
)
print("R2 Score from predicted x real %f" % r_value ** 2)


plt.scatter(y_test, test_ans)
plt.plot(y_test, y_test, "black", label="real values")
plt.plot(y_test, intercept + slope * y_test, "r", label="fitted line")
plt.xlabel("Real values")
plt.ylabel("Predicted values")

red_patch = mpatches.Patch(color="red", label="Regression from model")
black_patch = mpatches.Patch(color="black", label="Regression from real data")
plt.legend(handles=[red_patch, black_patch])

plt.savefig("../images/linear_reg_eval.png")  # save the generated figure
plt.show()

In [ ]:
i = 15
x_p = (test_ans[i] - intercept) / slope
print(X_test[i])
print(x_p)
print(y_test[i])